In [ ]:
import pyspark 
from pyspark import SparkConf, SparkContext

# Smoke Test

In [ ]:
conf = SparkConf()
conf.setMaster('local[4]')
conf.set("spark.files.maxPartitionBytes", "200K")
conf.set("spark.sql.files.maxPartitionBytes", "200K")
conf.set("spark.files.openCostInBytes", "1000")

disable_optimizations = True

if disable_optimizations:
    conf.set("spark.shuffle.sort.bypassMergeThreshold", "1")
    conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
    conf.set("spark.sql.adaptive.autoBroadcastJoinThreshold", "-1")
    conf.set("spark.sql.adaptive.coalescePartitions.enabled", "false")
    conf.set("spark.sql.adaptive.enabled", "false")
    conf.set("spark.sql.bucketing.coalesceBucketsInJoin.enabled", "false")
    conf.set("spark.sql.cbo.enabled", "false")
    conf.set("spark.sql.csv.filterPushdown.enabled", "false")
    conf.set("spark.sql.optimizer.dynamicPartitionPruning.enabled", "false")


sc = SparkContext(conf=conf)
sc.setLogLevel('INFO')
spark = pyspark.sql.SparkSession(sc)
sc

In [ ]:
!hostname
!echo
!ls /ext

In [ ]:
!ls /ext/dataset/*/* | shuf | head -n10  | sort

In [ ]:
rdd = sc.parallelize(range(1000))
rdd \
    .filter(lambda number: number % 10 == 0) \
    .takeSample(False, 10)

# Reading data

## Customers

In [ ]:
customers = spark.read \
    .option("header", True) \
    .csv("/ext/dataset/customers/*.csv")

In [ ]:
customers.show(5)

In [ ]:
customers.rdd.take(5)

In [ ]:
customers.rdd.getNumPartitions()

In [ ]:
customers.rdd.toDF().show(5)

In [ ]:
customers.count()

In [ ]:
customers.rdd.map(lambda row: type(row.customer_zip_code_prefix)).take(5)

## Products, Orders, Order-Items

In [ ]:
products = spark.read \
    .option("header", True) \
    .csv("/ext/dataset/products/*.csv")
products.show(5)
products.count()

In [ ]:
orders = spark.read \
    .option("header", True) \
    .csv("/ext/dataset/orders/*.csv")
orders.show(5)
orders.count()

In [ ]:
order_items = spark.read \
    .option("header", True) \
    .csv("/ext/dataset/order_items/*.csv")
order_items.show(5)
order_items.count()

# Count unique customers

In [ ]:
customers.count()

In [ ]:
customers.select("customer_id").distinct().count()

In [ ]:
customers.select("customer_unique_id").distinct().count()

# Spark API examples

## RDD API

In [ ]:
customers_with_most_orders = customers.rdd \
    .map(lambda row: (row.customer_unique_id, 1)) \
    .reduceByKey(lambda num_rows1, num_rows2: num_rows1 + num_rows2) \
    .sortBy(lambda customer_uid_and_num_rows: -customer_uid_and_num_rows[1]) \
    .take(10)
customers_with_most_orders

## DataFrame API

In [ ]:
home_products = products \
    .filter(products.product_category_name == 'cama_mesa_banho') \
    .filter(products.product_weight_g > 10000) \
    .select(["product_id", "product_category_name", "product_weight_g", (products.product_weight_g / 1000).alias('weight_kg') ])
home_products.show()

## SQL API

In [ ]:
home_products.createOrReplaceTempView("table_home_products")
order_items.createOrReplaceTempView("table_order_items")

ordered_perfumes_df = spark.sql("""
    SELECT 
        p.product_id product_id,
        o.order_id order_id,
        CAST (o.price AS float) price,
        p.weight_kg weight_kg
    FROM
        table_home_products p
        LEFT JOIN table_order_items o ON (p.product_id = o.product_id)    
    WHERE
        price < 200
    ORDER BY
        product_id
""")

ordered_perfumes_df.collect()[:15]

# Exercise 1

Find customers who ordered (`order_status == "delivered"`) at least 3 products from category *perfumaria* and return top 5 customeres who spend most money in that category.

**Please use (mix) all three (RDD, DataFrame, SQL) interfaces to compute the result.**

# Exercise 2

- duplicate notebook
- set `disable_optimizations` to `False`
- use "Restart Kernel and runn all cells" in both notebooks
- compare execution plans and computed DAGs

# Hints and references

- https://spark.apache.org/docs/latest/sql-getting-started.html - Spark SQL Guide
- https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html - PySpark SQL API reference
- https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations - Spark actions and transformations
- https://spark.apache.org/docs/latest/configuration.html#available-properties - Spark Configuration